# ___

# [ Machine Learning in Geosciences ]

**Department of Applied Geoinformatics and Carthography, Charles University** 

*Lukas Brodsky lukas.brodsky@natur.cuni.cz*

    
___



# End-to-End Machine Learning Project!


1/ Describe the Task

2/ Get and Explore the Data

3/ Prepare the Data for ML Algorithms!

4/ Select and Train a Model

5/ Fine-Tune the Model!

6/ Interpret the results

___    

# 1/ The Problem 

**1.1 Frame the problem:** 
The task is to build a model of housing prices in California using the California census data to be able to predict the median housing price in any district. 

**Assumptions of the problem:** 
* There exist some (most likely non-linear) relationship between input features (X) and the output target  variable (y); 
* the output target is a continuous variable, hence we employ regression type of model; 
* There are multiple features, hence multivariate regression; 
* There is no continuous flow of data -> batch learning;

**1.2 The expected result:** 
The developed model shall predict housing prices based set of characteristics (fetures) with error < 20 %. 


**Performance measures:** 
* RMSE or MAE; 
* How much we know about the data? -> Update after EDA. 



# Setup environment

In [ ]:
import os
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt 

# Sklearn 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestRegressor

# to make this notebook's output stable across runs
np.random.seed(42)
# To plot 'pretty' figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=12)
mpl.rc('xtick', labelsize=10)
mpl.rc('ytick', labelsize=10)

# SET PATH TO YOUR PROJECT DIRECTORY!
PROJECT_ROOT_DIR = "./"
if os.path.isdir(PROJECT_ROOT_DIR): 
    print('Ok continue.')
else: 
    print('Nok, set correct path to your project directory!')

# Set path to save project figures 
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="internal gelsd")

# 2/ Exploratory Data Analysis


In [ ]:
# Inport data 
HOUSING_PATH = os.path.join(PROJECT_ROOT_DIR)
# print(HOUSING_PATH)

# function to read the csv file 
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
# load data 
housing = load_housing_data()

# check header and some values 
housing.head()

In [ ]:
# evaluate the values 
housing.info()

In [ ]:
# Evaluate statistics of the features 
housing.describe()

In [ ]:
# features 
# housing_columns = list(housing.columns)
# housing_columns

### 2.1 Visualize the data to gain insights

In [ ]:
# histograms 
housing.hist(bins=50, figsize=(15,10))
# save_fig("attribute_histogram_plots") 

In [ ]:
# plot housing data (spatial plot )
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1,
             c="median_house_value",
             s=housing["population"]/100, label="population", figsize=(10,7),
             cmap=plt.get_cmap("jet"), colorbar=True)
plt.legend()
# save_fig("housing_prices_scatterplot")

### 2.2 Looking for Correlations

In [ ]:
# Remove non-numeric data 
housing_rval = housing.drop('ocean_proximity', axis=1)
# housing_rval.head()

In [ ]:
# Prepare correlation data 
corr_matrix = housing_rval.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# cross-scatterplot for selected features 
attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(7, 5), alpha=0.3)
# save_fig("scatter_matrix_plot")

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)
plt.axis([0, 16, 0, 550000])
# save_fig("income_vs_house_value_scatterplot")

### 2.3 Experiment with Attributes 
.. later on in data preparation. 

# 3. Data preparation

### 3.1 Prepare the data for Machine Learning algorithms - Feature Engineering

In [ ]:
housing.shape

### New fetures

In [ ]:
# add some more features (relative indicators) 
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
housing.columns

In [ ]:
housing.shape

In [ ]:
# Append new feature names to the list of columns 
housing_columns.append('rooms_per_household')
housing_columns.append('bedrooms_per_room')
housing_columns.append('population_per_household')
housing_columns

### Check correlations

In [ ]:
housing_rval = housing.drop('ocean_proximity', axis=1)

In [ ]:
corr_matrix = housing_rval.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

### 3.2 Fill no-data values or drop incomplete records

In [ ]:
# Which records have NaN record?
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

In [ ]:
sample_incomplete_rows.shape

### Drop NaN records 

In [ ]:
# Option 1: .dropna()
# sample_incomplete_rows.dropna(subset=["total_bedrooms"])    

In [ ]:
# Option 2: .drop()
# sample_incomplete_rows.drop("total_bedrooms", axis=1)       

### Fill-in meadian value 

In [ ]:
# Option 3: fill-in meadian value 
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) 
sample_incomplete_rows

### Sklearn Imputer

In [ ]:
try:
    from sklearn.impute import SimpleImputer # Scikit-Learn 0.20+
except ImportError:
    from sklearn.preprocessing import Imputer as SimpleImputer

imputer = SimpleImputer(strategy="median")

In [ ]:
# Remove the text attribute because median can only be calculated on numerical attributes:
housing_num = housing.drop('ocean_proximity', axis=1)
# housing_num

In [ ]:
# fit the imputer instance to the training data using the fit() method
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
# Check that this is the same as manually computing the median of each attribute:
housing_num.median().values

In [ ]:
# Now we can use this “trained” imputer to transform the training set 
# by replacing missing values by the learned medians

housing_num_imp = imputer.transform(housing_num)

# Numpy array containing the transformed features
print(type(housing_num_imp))

In [ ]:
# put it back into a Pandas DataFrame
housing_num_tr = pd.DataFrame(housing_num_imp, columns=housing_num.columns,
                          index=housing.index)

In [ ]:
# check the imputation 
housing_num_tr.loc[sample_incomplete_rows.index.values]

In [ ]:
imputer.strategy

In [ ]:
housing_num_tr.shape

In [ ]:
# Scale the numerical features: housing_num
# scaler = StandardScaler()
# scaler.fit(housing_num)
# scaler.mean_
# housing_num_scaled = scaler.transform(housing_num)

In [ ]:
# type(housing_num_scaled)

In [ ]:
# put it back into a Pandas DataFrame
# housing_num_scaled = pd.DataFrame(housing_num_scaled, columns=housing_num.columns,
#                           index=housing.index)

In [ ]:
# housing_num_scaled.head()

In [ ]:
# housing_num_scaled.shape

### 3.3 Harmonize numerical data
### Feature Endcoding 

Transform cathegorical text feature(s) into numerical. 

In [ ]:
try:
    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn > 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20

In [ ]:
housing["ocean_proximity"]

In [ ]:
housing["ocean_proximity"].shape 

In [ ]:
# encoder = OneHotEncoder()
encoder= OneHotEncoder(sparse_output=False).set_output(transform="pandas")
encoder.fit(housing[["ocean_proximity"]])
ocean_proximity_num = encoder.transform(housing[["ocean_proximity"]])

In [ ]:
type(ocean_proximity_num)

In [ ]:
ocean_proximity_num.head()

In [ ]:
housing_num_tr.shape

In [ ]:
ocean_proximity_num.shape

In [ ]:
# Merge the two DataFrames
housing_num_enc_merged = pd.concat([housing_num_tr, ocean_proximity_num], axis=1)
housing_num_enc_merged.head()

In [ ]:
housing_num_enc_merged.columns

In [ ]:
housing_num_enc_merged.shape

In [ ]:
# housing_columns.remove('ocean_proximity')

In [ ]:
# for item in list(ocean_proximity_num.columns):
#         housing_columns.append(item)

In [ ]:
# housing_columns

### 3.4 Split Data set into Training and Test Sets

Options: 

    * random sampling (introducing a significant sampling bias); 
    * stratified sampling (to ensure that split sample is representative of the whole populatio); 
    

### Numpy random solution

In [ ]:
# For illustration only. Sklearn has train_test_split()
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing_num_enc_merged, 0.5)
print(f'Train set size: {len(train_set)}, Test set size: {len(test_set)}')

### Scikit learn split functions

To make repeated experiments set `random_state` to some constatnt!

In [ ]:
# split 
train_set, test_set = train_test_split(housing_num_enc_merged, test_size=0.5, random_state=42)

In [ ]:
print(f'Train set size: {len(train_set)}, Test set size: {len(test_set)}')

### Stratified sampling


In [ ]:
# check histogram 
housing_num_enc_merged["median_income"].hist()

In [ ]:
# cut the histogram into bins 
housing_num_enc_merged["income_cat"] = pd.cut(housing_num_enc_merged["median_income"],
                               bins=[0.0, 2.0, 3.0, 4.0,  np.inf],
                                labels=[1, 2, 3, 4])

In [ ]:
# chaeck for NaNs 
housing_num_enc_merged["income_cat"].isnull().any()

In [ ]:
# housing_num_enc_merged["income_cat"]
# housing_num_enc_merged[housing_num_enc_merged["income_cat"].isnull().any(axis=1)].head()
# housing_num_enc_merged[housing_num_enc_merged["income_cat"].isnull()]['median_income'].hist()

In [ ]:
# check distribution in the bins
housing_num_enc_merged["income_cat"].hist()

In [ ]:
# housing_num_enc_merged["income_cat"] 

In [ ]:
# Do stratified sampling based on the income category!
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

for train_index, test_index in sss.split(housing_num_enc_merged, housing_num_enc_merged["income_cat"]):
    strat_train_set = housing_num_enc_merged.loc[train_index]
    strat_test_set = housing_num_enc_merged.loc[test_index]

In [ ]:
strat_train_set.head()

In [ ]:
# the income category proportions in the test set
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
# the income category proportions in the full housing dataset
housing_num_enc_merged["income_cat"].value_counts() / len(housing_num_enc_merged)

### Is there a balnce?

# 4. Select and train a model 

In [ ]:
# Prepare training data 
X_train = strat_train_set.drop('median_house_value', axis=1)
y_train = strat_train_set['median_house_value']

In [ ]:
# and testing data 
X_test = strat_test_set.drop('median_house_value', axis=1)
y_test = strat_test_set['median_house_value']

### Linear relationship? 

In [ ]:
# Model 
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

In [ ]:
# Evaluating: training MAE 
housing_predictions = lin_reg.predict(X_train)
lin_mae = mean_absolute_error(y_train, housing_predictions)
print(f'Linear model training MAE: {lin_mae}') 

In [ ]:
# Evaluating: testing MAE 
housing_test_predictions = lin_reg.predict(X_test)
lin_mae_test = mean_absolute_error(y_test, housing_test_predictions)
print(f'Linear model testing MAE: {lin_mae_test}') 

In [ ]:
print(f'Median relative error: {round(lin_mae / y_train.median() * 100., 2)} %') 

### Non-linear relationship? 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(X_train, y_train)

In [ ]:
# Evaluating: training MAE 
housing_predictions = tree_reg.predict(X_train)
tree_mae = mean_absolute_error(y_train, housing_predictions)
print(f'Tree model training MAE: {tree_mae}') 

In [ ]:
# Evaluating: testing MAE 
housing_test_predictions = tree_reg.predict(X_test)
tree_mae_test = mean_absolute_error(y_test, housing_test_predictions)
print(f'Tree model testing MAE: {tree_mae_test}') 

#### What does it mean? 

In [ ]:
print(f'Median relative error: {round(tree_mae / y_train.median() * 100., 2)} %') 

# Fine-tune the model

In [ ]:
# Try Random Forest 
forest_reg = RandomForestRegressor(random_state=42)

In [ ]:
# Combinations for Grid Search Cross-validation 
hyperparameter_space = {'n_estimators': [20, 50, 100], 
                        'max_depth': [10, 15, 20],
                        'min_samples_leaf': [2, 4]
                        }

gs = GridSearchCV(forest_reg, param_grid=hyperparameter_space, n_jobs=4,
                  scoring="neg_mean_absolute_error", cv=10, return_train_score=True)

In [ ]:
# run fit (be aware: TAKES LONG!)
gs.fit(X_train, y_train)

In [ ]:
# Optimal parameters and scores 
print("Optimal hyperparameter combination: ", gs.best_params_)

In [ ]:
feature_importances = gs.best_estimator_.feature_importances_

In [ ]:
# feature_importances
sorted(zip(feature_importances, list(X_train.columns)), reverse=True)

In [ ]:
gs.best_estimator_

In [ ]:
# Select final model and evaluate it with test set!
final_model = gs.best_estimator_

In [ ]:
print("Mean cross-validated accuracy of the best_estimator: ", round((-gs.best_score_), 2))

In [ ]:
# Evaluating: testing MAE 
housing_test_predictions = final_model.predict(X_test)
final_model_mae_test = mean_absolute_error(y_test, housing_test_predictions)
print(f'Final model testing MAE: {final_model_mae_test}') 

In [ ]:
print(f'Median relative error: {round(final_model_mae_test / y_train.median() * 100., 2)} %') 